In [1]:
# Import pandas to create DataFrame 
import pandas as pd

#Ignoring warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Make DataFrame of the given data 
train = pd.read_csv('train.csv')
valid = pd.read_csv('valid.csv')

In [3]:
train.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_763,feature_764,feature_765,feature_766,feature_767,feature_768,label_1,label_2,label_3,label_4
0,0.186257,-0.058807,0.024632,-0.163933,-0.146699,0.035889,0.111708,-0.162861,0.028249,-0.098063,...,0.055629,-0.010358,0.125754,0.011648,0.079197,0.093215,45,NaN,1,6
1,0.063431,-0.023597,0.068057,-0.252915,-0.061094,-0.027316,0.135747,-0.168147,0.091236,-0.078473,...,-0.014893,0.071721,0.018918,0.100032,-0.083042,0.088615,45,NaN,1,6
2,0.034962,0.035816,-0.029753,-0.094607,-0.017576,-0.053074,0.040121,-0.007932,0.097872,-0.024042,...,0.012415,0.015215,0.083808,0.031312,-0.056277,0.064702,45,NaN,1,6
3,0.033772,0.085612,0.067488,-0.073953,-0.180646,-0.024512,0.242879,-0.023374,-0.059999,0.002006,...,-0.078246,-0.032903,0.082949,-0.020659,0.082274,-0.050164,45,NaN,1,6
4,0.134305,0.062096,0.106920,-0.089327,0.117093,-0.077107,0.152579,0.047529,-0.015998,-0.110657,...,-0.094629,0.069718,0.014379,0.048124,0.007586,-0.016980,45,NaN,1,6


In [4]:
#Checking provided data
null_counts = train.isnull().sum()

print(null_counts)

feature_1        0
feature_2        0
feature_3        0
feature_4        0
feature_5        0
              ... 
feature_768      0
label_1          0
label_2        480
label_3          0
label_4          0
Length: 772, dtype: int64


In [5]:
# splitting dependent & Independent features
y = train[["label_1","label_2","label_3","label_4"]]
y_valid = valid[["label_1","label_2","label_3","label_4"]] 
x = train.drop(y, axis=1)
x_valid = valid.drop(y_valid, axis=1)

# Model Checking & Hyper parameter tuning

For KNN, I used Grid Search to do hyper parameter tuning

In [6]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.metrics import accuracy_score

def try_knn(train_x, train_y, vaid_x, valid_y):
    grid_params = { 'n_neighbors' : [3,5,7,15],
               'weights' : ['uniform','distance']}
    gs = GridSearchCV(KNeighborsClassifier(), grid_params, verbose = 1, cv=3, n_jobs = -1)
    g_res = gs.fit(train_x, train_y)
    print(f"Best Hyperparameters:",gs.best_params_)
    # Get the best k-NN model with the optimal hyperparameters
    best_knn = gs.best_estimator_
    # Evaluate the best model on the test data
    accuracy = best_knn.score(vaid_x, valid_y)
    print(f"Accuracy for KNN {accuracy}")      
        


Due to performance restriction of my machine, I do hyperparameter tuning manually for SVM.

In [26]:
from sklearn import svm

def try_svm(train_x, train_y, vaid_x, valid_y):
    clf = svm.SVC()
    clf.fit(train_x, train_y)
    X_valid_contiguous = np.ascontiguousarray(vaid_x)
    y_pred = clf.predict(X_valid_contiguous)
    print(f"Accuracy Score of SVM with default settings (RBF - exponential kernal) ", accuracy_score(valid_y, y_pred))
    
#     clf = svm.LinearSVC(dual="auto")
#     clf.fit(train_x, train_y)
#     X_valid_contiguous = np.ascontiguousarray(vaid_x)
#     y_pred = clf.predict(X_valid_contiguous)
#     print(f"Accuracy Score of Linear SVM (one-vs-the-rest) ", accuracy_score(valid_y, y_pred))
    
    clf = svm.SVC(kernel='linear')
    clf.fit(train_x, train_y)
    X_valid_contiguous = np.ascontiguousarray(vaid_x)
    y_pred = clf.predict(X_valid_contiguous)
    print(f"Accuracy Score of SVM with linear kernal function ", accuracy_score(valid_y, y_pred))
    
    clf = svm.SVC(kernel='sigmoid')
    clf.fit(train_x, train_y)
    X_valid_contiguous = np.ascontiguousarray(vaid_x)
    y_pred = clf.predict(X_valid_contiguous)
    print(f"Accuracy Score of SVM with sigmoid kernal function ", accuracy_score(valid_y, y_pred))
    
    clf = svm.SVC(kernel='poly',degree =2)
    clf.fit(train_x, train_y)
    X_valid_contiguous = np.ascontiguousarray(vaid_x)
    y_pred = clf.predict(X_valid_contiguous)
    print(f"Accuracy Score of SVM with polynomial kernal function with degree 2 ", accuracy_score(valid_y, y_pred))
    
    clf = svm.SVC(kernel='poly',degree =2)
    clf.fit(train_x, train_y)
    X_valid_contiguous = np.ascontiguousarray(vaid_x)
    y_pred = clf.predict(X_valid_contiguous)
    print(f"Accuracy Score of SVM with polynomial kernal function with degree 3 ", accuracy_score(valid_y, y_pred))  

Logistic Regression Model
Checking Hyper Parameter tuning for Logistic Regression

In [8]:
from sklearn.linear_model import LogisticRegression

def try_logistic(train_x, train_y, vaid_x, valid_y):
    for i in ["lbfgs","newton-cg","sag","saga"]:
        logistic_regression = LogisticRegression(multi_class='multinomial', solver='lbfgs')
        logistic_regression.fit(train_x,train_y)
        y_pred = logistic_regression.predict(vaid_x)
        print(f"Accuracy Score of Logistic Regression with solver {i}", accuracy_score(valid_y, y_pred))

Naive Bayes 
Trying out different distributions

In [9]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import ComplementNB
from sklearn.naive_bayes import CategoricalNB
from sklearn.naive_bayes import BernoulliNB

def try_naive(train_x, train_y, vaid_x, valid_y):
    
    min_value = np.min(train_x)
    scaled_train_x = train_x - min_value
    scaled_vaid_x = vaid_x - min_value
    
    clf = GaussianNB()
    clf.fit(scaled_train_x, train_y)
    y_pred = clf.predict(scaled_vaid_x)
    print(f"Accuracy Score of Naive Bayes with Gaussian Naive Bayes", accuracy_score(valid_y, y_pred))
    
    clf = MultinomialNB()
    clf.fit(scaled_train_x, train_y)
    y_pred = clf.predict(vaid_x)
    print(f"Accuracy Score of Naive Bayes with MultinomialNB", accuracy_score(valid_y, y_pred))
    
    clf = ComplementNB()
    clf.fit(scaled_train_x, train_y)
    y_pred = clf.predict(scaled_vaid_x)
    print(f"Accuracy Score of Naive Bayes with ComplementNB", accuracy_score(valid_y, y_pred))
    
    clf = CategoricalNB()
    clf.fit(scaled_train_x, train_y)
    y_pred = clf.predict(scaled_vaid_x)
    print(f"Accuracy Score of Naive Bayes with CategoricalNB", accuracy_score(valid_y, y_pred))
    
    clf = BernoulliNB()
    clf.fit(scaled_train_x, train_y)
    y_pred = clf.predict(scaled_vaid_x)
    print(f"Accuracy Score of Naive Bayes with BernoulliNB", accuracy_score(valid_y, y_pred))  
    



DecisionTree Classifier

In [10]:
from sklearn.tree import DecisionTreeClassifier

def try_decisionTree(train_x, train_y, vaid_x, valid_y):
    # Initialize and train the Decision Tree classifier
    clf = DecisionTreeClassifier()
    clf.fit(train_x, train_y)
    y_pred = clf.predict(vaid_x)
    print(f"Accuracy Score of Decision Tree", accuracy_score(valid_y, y_pred))

Random Forest

In [11]:
from sklearn.ensemble import RandomForestClassifier

def try_random_forest(train_x, train_y, vaid_x, valid_y):
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(train_x, train_y)
    y_pred = clf.predict(vaid_x)
    print(f"Accuracy Score of Random Forest", accuracy_score(valid_y, y_pred))

XG Boost

In [14]:
#installing XG Boost
!pip install xgboost

DEPRECATION: h2o-mlops-client main has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of h2o-mlops-client or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [15]:
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder


def try_xgBoost(train_x, train_y, vaid_x, valid_y):
    # Create an XGBoost classifier for multi-class classification
    clf = xgb.XGBClassifier(
    objective='multi:softmax',  # Set the objective for multi-class classification
    num_class=len(np.unique(y)),  # Number of classes
    random_state=42
    )
    le = LabelEncoder()
    train_y = le.fit_transform(train_y)
    clf.fit(train_x, train_y)
    y_pred = clf.predict(vaid_x)
    y_pred = le.inverse_transform(y_pred)
    print(f"Accuracy Score of XG Boost", accuracy_score(valid_y, y_pred))

# Feature Engineering Using Mutual Info Classification

In [16]:
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest

def select_cols_using_mutual_info_regression(x,y,n) :
    selected_columns = SelectKBest(mutual_info_classif, k=n)
    selected_columns.fit(x, y)
    return x.columns[selected_columns.get_support()]

In [17]:
selected_cols = select_cols_using_mutual_info_regression(x,y['label_1'],300)

In [54]:
#After reducing rows, I got poor validation accuracy, So I decided to not to use this method.
#x = x[selected_cols]
#x_valid = x_valid[selected_cols]

# Label 1

In [18]:
#Doing Hyper parameter tuning for KNN & checking the best accuracy for label 1
try_knn(x,y["label_1"],x_valid,y_valid["label_1"])

Fitting 3 folds for each of 8 candidates, totalling 24 fits
Best Hyperparameters: {'n_neighbors': 3, 'weights': 'distance'}
Accuracy for KNN 0.948


In [27]:
#Checking SVM Values for Label 1
try_svm(x,y["label_1"],x_valid,y_valid["label_1"])

Accuracy Score of SVM with default settings (RBF - exponential kernal)  0.956
Accuracy Score of SVM with linear kernal function  0.9746666666666667
Accuracy Score of SVM with sigmoid kernal function  0.8973333333333333
Accuracy Score of SVM with polynomial kernal function with degree 2  0.956
Accuracy Score of SVM with polynomial kernal function with degree 3  0.956


In [24]:
#Checking Logistic Regression Values for Label1. It provided very good results. 
try_logistic(x,y["label_1"],x_valid,y_valid["label_1"])

Accuracy Score of Logistic Regression with solver lbfgs 0.972
Accuracy Score of Logistic Regression with solver newton-cg 0.972
Accuracy Score of Logistic Regression with solver sag 0.972
Accuracy Score of Logistic Regression with solver saga 0.972


In [57]:
#It gave bad results
try_naive(x,y["label_1"],x_valid,y_valid["label_1"])

Accuracy Score of Naive Bayes with Gaussian Naive Bayes 0.62
Accuracy Score of Naive Bayes with MultinomialNB 0.17333333333333334
Accuracy Score of Naive Bayes with ComplementNB 0.376
Accuracy Score of Naive Bayes with CategoricalNB 0.204
Accuracy Score of Naive Bayes with BernoulliNB 0.013333333333333334


In [21]:
try_decisionTree(x,y["label_1"],x_valid,y_valid["label_1"])

Accuracy Score of Decision Tree 0.37333333333333335


In [22]:
try_random_forest(x,y["label_1"],x_valid,y_valid["label_1"])

Accuracy Score of Random Forest 0.9066666666666666


In [23]:
try_xgBoost(x,y["label_1"],x_valid,y_valid["label_1"])

Accuracy Score of XG Boost 0.9026666666666666


When considering validation accuracy results of label 1, Linear SVM and Logistic Regression gave good results, We can use Linear SVM model for our test data set.

In [36]:
# Test dataset
test = pd.read_csv('test.csv')
test = test.drop(["ID"], axis=1)
test.head()
# test["ID"]

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_759,feature_760,feature_761,feature_762,feature_763,feature_764,feature_765,feature_766,feature_767,feature_768
0,0.187868,-0.009268,0.039846,-0.102515,-0.109121,-0.079769,0.061295,-0.042890,0.099860,-0.066319,...,0.043647,0.061174,0.026771,-0.092734,-0.052383,-0.055526,0.029101,0.041857,-0.027872,0.099500
1,0.122119,0.019015,0.134483,0.007949,-0.157884,-0.033332,0.044334,-0.016869,-0.014088,-0.060542,...,-0.179798,-0.027611,0.240023,-0.005648,-0.057537,0.023922,-0.038126,-0.015171,-0.006270,-0.003790
2,0.079187,0.146195,-0.001316,-0.085596,-0.261093,-0.021620,0.268898,-0.103950,0.068976,-0.062388,...,-0.075460,0.037346,0.124254,-0.369695,-0.056773,-0.028471,0.019971,-0.065683,-0.047366,-0.121744
3,0.231196,0.021558,0.215534,-0.210258,-0.158189,0.041621,0.240254,-0.068112,-0.017550,-0.233767,...,0.083334,0.141642,-0.007219,0.031547,-0.010762,-0.169036,-0.030963,0.086698,0.114194,-0.036775
4,0.020412,0.063732,-0.065506,-0.089598,-0.130788,-0.018809,0.119304,-0.057494,0.094714,-0.047923,...,-0.001969,-0.003025,-0.021156,-0.136993,-0.003615,0.036371,0.048715,-0.002688,-0.016957,-0.004331


In [37]:
# taken model for this lable is 
# Accuracy Score of SVM with linear kernal function  0.9746666666666667
# try_logistic(x,y["label_1"],x_valid,y_valid["label_1"])
clf = svm.SVC(kernel='linear')

clf.fit(x, y["label_1"])
X_valid_contiguous = np.ascontiguousarray(x_valid)
y_pred = clf.predict(X_valid_contiguous)
print(f"Accuracy Score of SVM with linear kernal function ", accuracy_score(y_valid["label_1"], y_pred))

pridiction = clf.predict(test)
df = pd.DataFrame({'label_1': pridiction})

# Save the DataFrame as a CSV file
df.to_csv('label_1.csv', index=False)
print(pridiction)


Accuracy Score of SVM with linear kernal function  0.9746666666666667
[26 18 16  7 58 46  7 22 29 26 33  6 54 51 29 48 23  2 43 42 11 35 39 36
 44 48 28 55  2 51 55 46  1 56 30 22 50  6 40 18 19 47  6 46 59 25 31 38
  5 32 25 56 39 37 32 29 46 52 28 34 16 47 55 24 23 18 20  6 36 38 22 28
 44 46 55 59 49 55 57 12 52 38 29  4 31 49 14 19 12 45 36 43 34 32 54 46
 27  5  7 42 19 26 31 36 53 10 12 33 38  2 57 37 17 50 46 46 17 34 32 40
 19 40  8 22 38 15 39  9 52 37 28 13 10 55 28 25 42 57 16 27 23  3 51 27
 32 46 16 53 11 46 42 37 47 50  3 41 19 20 15  8  3 18  5 28 46 49 16 40
 17 20 53 57  7 16 11 35 34 52 41 55  7 18 21 19 37 55 29 28 23 35 46 52
 30 27 13  7 28 14  3 36 17 60 18  3 22  9 46 25 17 26 19 36 36 21  1 44
 39 47 38  6 50 39 23 37 48 22 58 15 30 38 18 54  3 41 16  8 43 28  8 57
 10 16  4 20 15 51 56 33 16 33 50 51  5 51 31  6  9 31 57 22  5 11 11 43
  8  7 58 50 50 58 13 13 47 50 60 50 47 36 28 36 49  7 34 32  5  2 19 55
 56 50 30 32 41 37 41 51 20 20 11 21 44 31 15 40 46 48

# Label 3

In [38]:
try_knn(x,y["label_3"],x_valid,y_valid["label_3"])

Fitting 3 folds for each of 8 candidates, totalling 24 fits
Best Hyperparameters: {'n_neighbors': 7, 'weights': 'distance'}
Accuracy for KNN 0.9973333333333333


In [39]:
try_logistic(x,y["label_3"],x_valid,y_valid["label_3"])

Accuracy Score of Logistic Regression with solver lbfgs 0.9986666666666667
Accuracy Score of Logistic Regression with solver newton-cg 0.9986666666666667
Accuracy Score of Logistic Regression with solver sag 0.9986666666666667
Accuracy Score of Logistic Regression with solver saga 0.9986666666666667


In [40]:
try_svm(x,y["label_3"],x_valid,y_valid["label_3"])

Accuracy Score of SVM with default settings (RBF - exponential kernal)  0.9986666666666667
Accuracy Score of SVM with linear kernal function  0.9986666666666667
Accuracy Score of SVM with sigmoid kernal function  0.9346666666666666
Accuracy Score of SVM with polynomial kernal function with degree 2  0.9986666666666667
Accuracy Score of SVM with polynomial kernal function with degree 3  0.9986666666666667


In [41]:
try_naive(x,y["label_3"],x_valid,y_valid["label_3"])

Accuracy Score of Naive Bayes with Gaussian Naive Bayes 0.8653333333333333
Accuracy Score of Naive Bayes with MultinomialNB 0.8266666666666667
Accuracy Score of Naive Bayes with ComplementNB 0.8973333333333333
Accuracy Score of Naive Bayes with CategoricalNB 0.84
Accuracy Score of Naive Bayes with BernoulliNB 0.8146666666666667


In [42]:
try_decisionTree(x,y["label_3"],x_valid,y_valid["label_3"])

Accuracy Score of Decision Tree 0.92


In [43]:
try_random_forest(x,y["label_3"],x_valid,y_valid["label_3"])

Accuracy Score of Random Forest 0.9613333333333334


In [44]:
try_xgBoost(x,y["label_3"],x_valid,y_valid["label_3"])

Accuracy Score of XG Boost 0.9906666666666667


When considering validation accuracy results of label 3, KNN, Logistic Regression, SVM and XG Boost provided good accuracy. Since we got good accuracy close to 100%, We can use SVM model for our test data set.

In [47]:
# taken model for this lable is 
# Accuracy Score of SVM with linear kernal function  0.9746666666666667
# try_knn(x,y["label_3"],x_valid,y_valid["label_3"])
# Fitting 3 folds for each of 8 candidates, totalling 24 fits
# Best Hyperparameters: {'n_neighbors': 7, 'weights': 'distance'}
# Accuracy for KNN 0.9973333333333333

grid_params = { 'n_neighbors' : [3,5,7,15],
               'weights' : ['uniform','distance']}
gs = GridSearchCV(KNeighborsClassifier(), grid_params, verbose = 1, cv=3, n_jobs = -1)
g_res = gs.fit(x, y["label_3"])
print(f"Best Hyperparameters:",gs.best_params_)
# Get the best k-NN model with the optimal hyperparameters
best_knn = gs.best_estimator_
# Evaluate the best model on the test data
accuracy = best_knn.score(x_valid,y_valid["label_3"])
print(f"Accuracy for KNN {accuracy}")     

pridiction = best_knn.predict(test)
df = pd.DataFrame({'label_3': pridiction})

# Save the DataFrame as a CSV file
df.to_csv('label_3.csv', index=False)
print(pridiction)


Fitting 3 folds for each of 8 candidates, totalling 24 fits
Best Hyperparameters: {'n_neighbors': 7, 'weights': 'distance'}
Accuracy for KNN 0.9973333333333333
[0 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1
 1 1 1 1 0 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 0 0 1 1 0 1 1 1 1 1 1 0 1 1 0 1 1
 1 0 1 1 0 0 0 1 1 1 1 1 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 0 1 0 1 1 0 1 1 1 0
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1
 1 1 0 1 1 1 0 1 1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 0 1 1 1 1
 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0
 1 1 1 1 0 1 1 0 1 1 0 1 1 0 1 0 1 0 0 0 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 0 1 1 1 1 0 1 1 1 1 0 1 0 1 1 1 1
 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 0 1 1 1 1 0 1 1
 1 0 1 1 1 0 1 1 1 0 0 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
 1 0 1 1 1 1 1 

# Label 4

# Handling Class Imbalance using SMOTE 

In [52]:
y["label_4"].value_counts()

6     19938
2      1449
0       955
12      954
7       938
13      482
1       481
11      480
10      480
3       479
5       478
9       472
4       469
8       465
Name: label_4, dtype: int64

In [53]:
pip install imbalanced-learn


DEPRECATION: h2o-mlops-client main has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of h2o-mlops-client or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [50]:
#Since 6 value affects class balance property. We decided to undersample it.
from imblearn.over_sampling  import SMOTE
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = smote.fit_resample(x, y["label_4"])

In [51]:
y_resampled.value_counts()
#However resampled values gave poor performance. So I avoided using them.

6     19938
13    19938
4     19938
5     19938
1     19938
2     19938
7     19938
3     19938
0     19938
12    19938
9     19938
8     19938
11    19938
10    19938
Name: label_4, dtype: int64

In [54]:
try_knn(x,y["label_4"],x_valid,y_valid["label_4"])

Fitting 3 folds for each of 8 candidates, totalling 24 fits
Best Hyperparameters: {'n_neighbors': 7, 'weights': 'distance'}
Accuracy for KNN 0.9626666666666667


In [55]:
try_logistic(x,y["label_4"],x_valid,y_valid["label_4"])

Accuracy Score of Logistic Regression with solver lbfgs 0.9306666666666666
Accuracy Score of Logistic Regression with solver newton-cg 0.9306666666666666
Accuracy Score of Logistic Regression with solver sag 0.9306666666666666
Accuracy Score of Logistic Regression with solver saga 0.9306666666666666


In [56]:
try_svm(x,y["label_4"],x_valid,y_valid["label_4"])

Accuracy Score of SVM with default settings (RBF - exponential kernal)  0.9426666666666667
Accuracy Score of SVM with linear kernal function  0.9573333333333334
Accuracy Score of SVM with sigmoid kernal function  0.7653333333333333
Accuracy Score of SVM with polynomial kernal function with degree 2  0.9413333333333334
Accuracy Score of SVM with polynomial kernal function with degree 3  0.9413333333333334


In [57]:
try_naive(x,y["label_4"],x_valid,y_valid["label_4"])

Accuracy Score of Naive Bayes with Gaussian Naive Bayes 0.36533333333333334
Accuracy Score of Naive Bayes with MultinomialNB 0.7093333333333334
Accuracy Score of Naive Bayes with ComplementNB 0.536
Accuracy Score of Naive Bayes with CategoricalNB 0.6933333333333334
Accuracy Score of Naive Bayes with BernoulliNB 0.708


In [58]:
try_decisionTree(x,y["label_4"],x_valid,y_valid["label_4"])

Accuracy Score of Decision Tree 0.6973333333333334


In [59]:
try_random_forest(x,y["label_4"],x_valid,y_valid["label_4"])

Accuracy Score of Random Forest 0.7666666666666667


In [60]:
try_xgBoost(x,y["label_4"],x_valid,y_valid["label_4"])

Accuracy Score of XG Boost 0.92


When considering validation accuracy results of label 4, KNN, Logistic Regression and SVM  provided good accuracy. We can use ensambled method consisting above classifiers to predict values for our test data. 

In [61]:
# taken model for this lable is 
# Accuracy Score of SVM with linear kernal function  0.9746666666666667
# try_knn(x,y["label_3"],x_valid,y_valid["label_3"])
# Fitting 3 folds for each of 8 candidates, totalling 24 fits
# Best Hyperparameters: {'n_neighbors': 7, 'weights': 'distance'}
# Accuracy for KNN 0.9626666666666667

grid_params = { 'n_neighbors' : [3,5,7,15],
               'weights' : ['uniform','distance']}
gs = GridSearchCV(KNeighborsClassifier(), grid_params, verbose = 1, cv=3, n_jobs = -1)
g_res = gs.fit(x, y["label_4"])
print(f"Best Hyperparameters:",gs.best_params_)
# Get the best k-NN model with the optimal hyperparameters
best_knn = gs.best_estimator_
# Evaluate the best model on the test data
accuracy = best_knn.score(x_valid,y_valid["label_4"])
print(f"Accuracy for KNN {accuracy}")     

pridiction = best_knn.predict(test)
df = pd.DataFrame({'label_4': pridiction})

# Save the DataFrame as a CSV file
df.to_csv('label_4.csv', index=False)
print(pridiction)


Fitting 3 folds for each of 8 candidates, totalling 24 fits
Best Hyperparameters: {'n_neighbors': 7, 'weights': 'distance'}
Accuracy for KNN 0.9626666666666667
[ 2  8  6  6  6  6  6  6  6  2  6  6  6  6  6  6  6  6  5  0  6  2  6  6
  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  8  6  3  6  6  6  1  6 12
  6  0  1  6  6  7  0  6  6  5  6  6  6  3  6  2  6  6  6  6  6 12  6  6
  6  6  6  6  6  6  6  6  5 12  6  6  6  6 12  4  6  6  6  6  6  0  6  6
  6  6  6  0  4  2  6  6  6  6  6  6 12  6  6  7  6  6  6  6  6  6  6  6
  6  6  6  6 12  9  6 11  5  6  6  6  6  6  6  1  0  6  6  7  6  6  6  7
  0  6  6  6  6  6  0  7  3  6  6 10  4  6  9  6  6  8  6  6  6  6  6  6
  6  6  6  6  7  6  6  6  6  5  9  6  6  8  6  4  7  6  6  6  6  2  6  5
  6  7  6  6  6 12  6  6  6 13  8  6  6 11  6  1  6  2  6  6  6  6  6  6
  6  3 12  6  6  6  6  7  6  6  6  9  6 12  8  6  6 10  6  6  6  6  6  5
  6  6  6  6  9  6  6  6  6  6  6  6  6  6  6  6 11  6  6  6  6  6  6  6
  6  6  6  6  6  6  6  6  3  6 13  6 

# Label 2

# Handling Missing Values 

Since i am handling each feature separately, I thought removing the rows which have empty values for label 2, is the best way to handle the missing values for label 2 as no artifical entries will not be added.

In [62]:
data_label_2 = train.dropna(subset=['label_2'])
valid_label_2 = valid.dropna(subset=['label_2'])
y_label_2 = data_label_2[["label_1","label_2","label_3","label_4"]] 
x_label_2 = data_label_2.drop(y_label_2, axis=1)
y_label_2_valid = valid_label_2[["label_1","label_2","label_3","label_4"]] 
x_label_2_valid = valid_label_2.drop(y_label_2_valid, axis=1)

In [66]:
try_knn(x_label_2,y_label_2["label_2"],x_label_2_valid,y_label_2_valid["label_2"])

Fitting 3 folds for each of 8 candidates, totalling 24 fits
Best Hyperparameters: {'n_neighbors': 3, 'weights': 'distance'}
Accuracy for KNN 0.9470108695652174


In [67]:
try_logistic(x_label_2,y_label_2["label_2"],x_label_2_valid,y_label_2_valid["label_2"])

Accuracy Score of Logistic Regression with solver lbfgs 0.7540760869565217
Accuracy Score of Logistic Regression with solver newton-cg 0.7540760869565217
Accuracy Score of Logistic Regression with solver sag 0.7540760869565217
Accuracy Score of Logistic Regression with solver saga 0.7540760869565217


In [68]:
try_svm(x_label_2,y_label_2["label_2"],x_label_2_valid,y_label_2_valid["label_2"])

Accuracy Score of SVM with default settings (RBF - exponential kernal)  0.8804347826086957
Accuracy Score of SVM with linear kernal function  0.8546195652173914
Accuracy Score of SVM with sigmoid kernal function  0.485054347826087
Accuracy Score of SVM with polynomial kernal function with degree 2  0.873641304347826
Accuracy Score of SVM with polynomial kernal function with degree 3  0.873641304347826


In [69]:
try_naive(x_label_2,y_label_2["label_2"],x_label_2_valid,y_label_2_valid["label_2"])

Accuracy Score of Naive Bayes with Gaussian Naive Bayes 0.34782608695652173
Accuracy Score of Naive Bayes with MultinomialNB 0.1956521739130435
Accuracy Score of Naive Bayes with ComplementNB 0.328804347826087


IndexError: index 1 is out of bounds for axis 1 with size 1

In [70]:
try_decisionTree(x_label_2,y_label_2["label_2"],x_label_2_valid,y_label_2_valid["label_2"])

Accuracy Score of Decision Tree 0.375


In [71]:
try_random_forest(x_label_2,y_label_2["label_2"],x_label_2_valid,y_label_2_valid["label_2"])

Accuracy Score of Random Forest 0.782608695652174


In [72]:
try_xgBoost(x_label_2,y_label_2["label_2"],x_label_2_valid,y_label_2_valid["label_2"])

Accuracy Score of XG Boost 0.8872282608695652


Here KNN model performed well beating other models. So here we can use KNN model to predict label 2 data points.

In [74]:
# taken model for this lable is 
# Accuracy Score of SVM with linear kernal function  0.9746666666666667
# try_knn(x_label_2,y_label_2["label_2"],x_label_2_valid,y_label_2_valid["label_2"])
# Fitting 3 folds for each of 8 candidates, totalling 24 fits
# Best Hyperparameters: {'n_neighbors': 3, 'weights': 'distance'}
# Accuracy for KNN 0.9470108695652174



grid_params = { 'n_neighbors' : [3,5,7,15],
               'weights' : ['uniform','distance']}
gs = GridSearchCV(KNeighborsClassifier(), grid_params, verbose = 1, cv=3, n_jobs = -1)
g_res = gs.fit(x_label_2, y_label_2["label_2"])
print(f"Best Hyperparameters:",gs.best_params_)
# Get the best k-NN model with the optimal hyperparameters
best_knn = gs.best_estimator_
# Evaluate the best model on the test data
accuracy = best_knn.score(x_label_2_valid,y_label_2_valid["label_2"])
print(f"Accuracy for KNN {accuracy}")     

pridiction = best_knn.predict(test)
df = pd.DataFrame({'label_2': pridiction})

# Save the DataFrame as a CSV file
df.to_csv('label_2.csv', index=False)
print(pridiction)


Fitting 3 folds for each of 8 candidates, totalling 24 fits
Best Hyperparameters: {'n_neighbors': 3, 'weights': 'distance'}
Accuracy for KNN 0.9470108695652174
[22. 25. 30. 27. 29. 30. 27. 33. 23. 22. 26. 25. 27. 26. 23. 26. 28. 25.
 22. 29. 33. 24. 29. 22. 61. 26. 28. 23. 25. 26. 23. 30. 30. 24. 28. 28.
 24. 25. 26. 25. 26. 23. 25. 30. 31. 22. 26. 32. 25. 23. 22. 24. 29. 27.
 23. 23. 30. 34. 28. 25. 30. 23. 23. 26. 28. 28. 25. 25. 22. 32. 33. 28.
 61. 30. 23. 31. 26. 26. 27. 26. 34. 32. 23. 23. 26. 26. 31. 23. 26. 29.
 29. 31. 25. 23. 27. 30. 31. 25. 27. 29. 23. 22. 26. 22. 24. 36. 26. 26.
 32. 25. 27. 27. 26. 24. 30. 30. 26. 25. 25. 26. 26. 26. 41. 33. 32. 28.
 29. 35. 34. 25. 28. 27. 36. 23. 28. 22. 29. 27. 30. 31. 23. 25. 26. 31.
 23. 30. 30. 33. 33. 30. 29. 27. 23. 24. 31. 30. 23. 25. 28. 23. 31. 25.
 25. 28. 30. 26. 30. 26. 26. 25. 23. 27. 31. 30. 33. 24. 25. 34. 28. 23.
 27. 25. 26. 23. 27. 23. 29. 28. 28. 24. 30. 34. 28. 31. 27. 25. 28. 31.
 31. 22. 26. 27. 25. 31. 33. 35. 30. 

## Combine all CSV files for submission

In [144]:
label_1_o = pd.read_csv('label_1.csv', header=None)[1:]
label_2_o = pd.read_csv('label_2.csv', header=None)[1:]
label_3_o = pd.read_csv('label_3.csv', header=None)[1:]
label_4_o = pd.read_csv('label_4.csv', header=None)[1:]

In [145]:
label_3_o

,0
1,0
2,1
3,1
4,1
5,0
...,...
740,1
741,1
742,1
743,1


In [155]:
combined_df = pd.concat([label_1_o, label_2_o, label_3_o, label_4_o], axis=1)

column_names = ["label_1", "label_2", "label_3", "label_4"]  # Replace with your desired column names
combined_df.columns = column_names

combined_df['ID'] = range(1, len(combined_df) + 1)

column_order = ['ID',"label_1", "label_2", "label_3", "label_4"]

# Use reindex to reorder the columns
combined_df = combined_df.reindex(columns=column_order)

combined_df.to_csv("combined_data.csv",index=False)
combined_df

,ID,label_1,label_2,label_3,label_4
1,1,26,22.0,0,2
2,2,18,25.0,1,8
3,3,16,30.0,1,6
4,4,7,27.0,1,6
5,5,58,29.0,0,6
...,...,...,...,...,...
740,740,46,30.0,1,6
741,741,35,24.0,1,2
742,742,54,27.0,1,6
743,743,38,32.0,1,12


In [156]:
combined_csv = pd.read_csv("combined_data.csv")
combined_csv.head()

,ID,label_1,label_2,label_3,label_4
0,1,26,22.0,0,2
1,2,18,25.0,1,8
2,3,16,30.0,1,6
3,4,7,27.0,1,6
4,5,58,29.0,0,6


In [157]:
combined_csv

,ID,label_1,label_2,label_3,label_4
0,1,26,22.0,0,2
1,2,18,25.0,1,8
2,3,16,30.0,1,6
3,4,7,27.0,1,6
4,5,58,29.0,0,6
...,...,...,...,...,...
739,740,46,30.0,1,6
740,741,35,24.0,1,2
741,742,54,27.0,1,6
742,743,38,32.0,1,12
